In [366]:
class Stock:
    def __init__(self, symbol, date, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume

    def __repr__(self):
        return f"Stock(symbol={self.symbol}, date={self.date}, open={self.open}, high={self.high}, low={self.low}, close={self.close}, volume={self.volume})"
    
    def toJSON(self):
        return dict(object="Stock",
                    symbol=self.symbol,
                    date=self.date,
                    open=self.open,
                    high=self.high,
                    low=self.low,
                    close=self.close,
                    volume=self.volume)
    
    def __eq__(self, other):
        return isinstance(other, Stock) and self.toJSON() == other.toJSON()


class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.volume = volume
        self.commission = commission

    def __repr__(self):
        return f"Trade(symbol={self.symbol}, timestamp={self.timestamp}, order={self.order}, price={self.price}, commission={self.commission}, volume={self.volume})"

    def toJSON(self):
        return dict(object="Trade",
                    symbol=self.symbol,
                    timestamp=self.timestamp,
                    order=self.order,
                    price=self.price,
                    volume=self.volume,
                    commission=self.commission)
    
    def __eq__(self, other):
        return isinstance(other, Trade) and self.toJSON() == other.toJSON()

In [367]:


from datetime import date, datetime
from decimal import Decimal

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22), 
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22), 
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22), 
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],
    
    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [368]:
from pprint import pprint

class CustomJSONEncoder(json.JSONEncoder):
    def __init__(self, *args, **kwargs):
        super().__init__(indent=2)
    
    def default(self, arg):
        if isinstance(arg, date) or isinstance(arg, datetime):
            return arg.isoformat()
        elif isinstance(arg, Decimal):
            return dict(object="Decimal", value=str(arg))
        elif isinstance(arg, Stock) or isinstance(arg, Trade):
            return arg.toJSON()
        else:
            super().default(arg)

In [369]:
enc = json.dumps(activity, cls=CustomJSONEncoder)
print(enc)

{
  "quotes": [
    {
      "object": "Stock",
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": {
        "object": "Decimal",
        "value": "338.19"
      },
      "high": {
        "object": "Decimal",
        "value": "338.64"
      },
      "low": {
        "object": "Decimal",
        "value": "337.60"
      },
      "close": {
        "object": "Decimal",
        "value": "338.19"
      },
      "volume": 365607
    },
    {
      "object": "Stock",
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": {
        "object": "Decimal",
        "value": "176.66"
      },
      "high": {
        "object": "Decimal",
        "value": "177.25"
      },
      "low": {
        "object": "Decimal",
        "value": "176.64"
      },
      "close": {
        "object": "Decimal",
        "value": "176.78"
      },
      "volume": 3699184
    },
    {
      "object": "Stock",
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": {
        "object": 

In [370]:
def custom_decoder(arg):
    if "object" in arg:
        if arg["object"] == "Stock":
            return Stock(symbol=arg["symbol"], date=arg["date"], open_=arg["open"], high=arg["high"], low=arg["low"], close=arg["close"], volume=arg["volume"])
        elif arg["object"] == "Decimal":
            return Decimal(arg["value"])
        elif arg["object"] == "Trade":
            return Trade(symbol=arg["symbol"], timestamp=arg["timestamp"], order=arg["order"], price=arg["price"], commission=arg["commission"], volume=arg["volume"])
    else:
        return arg

In [371]:
json.loads(enc)

{'quotes': [{'object': 'Stock',
   'symbol': 'TSLA',
   'date': '2018-11-22',
   'open': {'object': 'Decimal', 'value': '338.19'},
   'high': {'object': 'Decimal', 'value': '338.64'},
   'low': {'object': 'Decimal', 'value': '337.60'},
   'close': {'object': 'Decimal', 'value': '338.19'},
   'volume': 365607},
  {'object': 'Stock',
   'symbol': 'AAPL',
   'date': '2018-11-22',
   'open': {'object': 'Decimal', 'value': '176.66'},
   'high': {'object': 'Decimal', 'value': '177.25'},
   'low': {'object': 'Decimal', 'value': '176.64'},
   'close': {'object': 'Decimal', 'value': '176.78'},
   'volume': 3699184},
  {'object': 'Stock',
   'symbol': 'MSFT',
   'date': '2018-11-22',
   'open': {'object': 'Decimal', 'value': '103.25'},
   'high': {'object': 'Decimal', 'value': '103.48'},
   'low': {'object': 'Decimal', 'value': '103.07'},
   'close': {'object': 'Decimal', 'value': '103.11'},
   'volume': 4493689}],
 'trades': [{'object': 'Trade',
   'symbol': 'TSLA',
   'timestamp': '2018-11-22T

In [376]:
import re

class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        pattern_decimal = r'"object"\s*:\s*"Decimal"'
        if re.search(pattern_decimal, arg):
            obj = self.make_decimal(obj)
        pattern_stock = r'"object"\s*:\s*"Stock"'
        if re.search(pattern_stock, arg):
            obj = self.make_stock(obj)
        pattern_trade = r'"object"\s*:\s*"Trade"'
        if re.search(pattern_trade, arg):
            obj = self.make_trade(obj)
        

        return obj
    
    
    def make_stock(self, obj):
        if isinstance(obj, dict):
            if obj.get("object", None) == "Stock":
                obj = Stock(obj["symbol"], datetime.strptime(obj["date"], "%Y-%m-%d").date(), obj["open"], obj["high"], obj["low"], obj["close"], obj["volume"])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_stock(value)

        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_stock(item)

        return obj
    
    def make_trade(self, obj):
        if isinstance(obj, dict):
            if obj.get("object", None) == "Trade":
                obj = Trade(obj["symbol"], datetime.strptime(obj["timestamp"], "%Y-%m-%dT%H:%M:%S"), obj["order"], obj["price"], obj["volume"], obj["commission"])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_trade(value)

        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_trade(item)

        return obj
    
    def make_decimal(self, obj):
        if isinstance(obj, dict):
            if obj.get("object", None) == "Decimal":
                obj = Decimal(obj["value"])
            else:
                for key, value in obj.items():
                    obj[key] = self.make_decimal(value)
        
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.make_decimal(item)

        return obj

In [377]:
decoded = json.loads(enc, cls=CustomDecoder)
decoded

{'quotes': [Stock(symbol=TSLA, date=2018-11-22, open=338.19, high=338.64, low=337.60, close=338.19, volume=365607),
  Stock(symbol=AAPL, date=2018-11-22, open=176.66, high=177.25, low=176.64, close=176.78, volume=3699184),
  Stock(symbol=MSFT, date=2018-11-22, open=103.25, high=103.48, low=103.07, close=103.11, volume=4493689)],
 'trades': [Trade(symbol=TSLA, timestamp=2018-11-22 10:05:12, order=buy, price=338.25, commission=9.99, volume=100),
  Trade(symbol=AAPL, timestamp=2018-11-22 10:30:05, order=sell, price=177.01, commission=9.99, volume=20)]}

In [380]:
activity

{'quotes': [Stock(symbol=TSLA, date=2018-11-22, open=338.19, high=338.64, low=337.60, close=338.19, volume=365607),
  Stock(symbol=AAPL, date=2018-11-22, open=176.66, high=177.25, low=176.64, close=176.78, volume=3699184),
  Stock(symbol=MSFT, date=2018-11-22, open=103.25, high=103.48, low=103.07, close=103.11, volume=4493689)],
 'trades': [Trade(symbol=TSLA, timestamp=2018-11-22 10:05:12, order=buy, price=338.25, commission=9.99, volume=100),
  Trade(symbol=AAPL, timestamp=2018-11-22 10:30:05, order=sell, price=177.01, commission=9.99, volume=20)]}

In [381]:
decoded

{'quotes': [Stock(symbol=TSLA, date=2018-11-22, open=338.19, high=338.64, low=337.60, close=338.19, volume=365607),
  Stock(symbol=AAPL, date=2018-11-22, open=176.66, high=177.25, low=176.64, close=176.78, volume=3699184),
  Stock(symbol=MSFT, date=2018-11-22, open=103.25, high=103.48, low=103.07, close=103.11, volume=4493689)],
 'trades': [Trade(symbol=TSLA, timestamp=2018-11-22 10:05:12, order=buy, price=338.25, commission=9.99, volume=100),
  Trade(symbol=AAPL, timestamp=2018-11-22 10:30:05, order=sell, price=177.01, commission=9.99, volume=20)]}

In [277]:
activity.keys()

dict_keys(['quotes', 'trades'])

In [303]:
json.loads(enc)

{'quotes': [{'object': 'Stock',
   'symbol': 'TSLA',
   'date': '2018-11-22',
   'open': {'object': 'Decimal', 'value': '338.19'},
   'high': {'object': 'Decimal', 'value': '338.64'},
   'low': {'object': 'Decimal', 'value': '337.60'},
   'close': {'object': 'Decimal', 'value': '338.19'},
   'volume': 365607},
  {'object': 'Stock',
   'symbol': 'AAPL',
   'date': '2018-11-22',
   'open': {'object': 'Decimal', 'value': '176.66'},
   'high': {'object': 'Decimal', 'value': '177.25'},
   'low': {'object': 'Decimal', 'value': '176.64'},
   'close': {'object': 'Decimal', 'value': '176.78'},
   'volume': 3699184},
  {'object': 'Stock',
   'symbol': 'MSFT',
   'date': '2018-11-22',
   'open': {'object': 'Decimal', 'value': '103.25'},
   'high': {'object': 'Decimal', 'value': '103.48'},
   'low': {'object': 'Decimal', 'value': '103.07'},
   'close': {'object': 'Decimal', 'value': '103.11'},
   'volume': 4493689}],
 'trades': [{'object': 'Trade',
   'symbol': 'TSLA',
   'timestamp': '2018-11-22T